# Custom models

In [1]:
import brian2 as b
from brian2.units import Hz, ms, mV, nS, pF, pA

from dendrify import Dendrite, NeuronModel, Soma

b.prefs.codegen.target = 'numpy'  # faster for simple simulations

ModuleNotFoundError: No module named 'dendrify'

In [2]:
def create_neuron_model(soma, dend):
    model = NeuronModel([(soma, dend, 15*nS)], v_rest=-60*mV)
    model.add_params({
        'Vth': -40*mV,
        'tauw_soma': 150*ms,
        'a_soma': 0*nS,
        'b_soma': 50*pA,
        'Vr': -50*mV
    })
    return model

def create_neuron_group(model):
    return model.make_neurongroup(
        1, method='euler',
        threshold='V_soma > Vth',
        reset='V_soma = Vr; w_soma += b_soma',
        refractory=4*ms
    )

def create_brian_objects(neuron):
    poisson = b.PoissonGroup(10, rates=20*Hz)
    syn = b.Synapses(poisson, neuron, on_pre='s_AMPA_x_dend += 1; s_NMDA_x_dend += 1')
    syn.connect(p=1)
    mon = b.StateMonitor(neuron, ['V_soma', 'V_dend', 'w_soma'], record=True)
    return poisson, syn, mon

def run_simulation(net):
    b.start_scope()
    b.seed(42)
    net.run(500*ms)

def plot_results(mon):
    time = mon.t / ms
    v_soma = mon.V_soma[0] / mV
    v_dend = mon.V_dend[0] / mV
    w = mon.w_soma[0] / pA

    fig, (ax1, ax2) = b.subplots(2, 1, figsize=[6, 6], sharex=True)
    ax1.plot(time, v_soma, label='soma')
    ax1.plot(time, v_dend, label='dendrite', c='C3')
    ax1.set_ylabel('Voltage (mV)')
    ax1.legend()
    ax2.plot(time, w, color='black', label='w')
    ax2.set_ylabel('Adaptation current (pA)')
    ax2.set_xlabel('Time (ms)')
    ax2.legend()
    fig.tight_layout()
    b.show()

def one_function_to_rule_them_all(soma, dend):
    model = create_neuron_model(soma, dend)
    neuron = create_neuron_group(model)
    poisson, syn, mon = create_brian_objects(neuron)
    net = b.Network(neuron, poisson, syn, mon)
    run_simulation(net)
    plot_results(mon)

In [3]:
soma = Soma('soma', model='adaptiveIF', cm_abs=200*pF, gl_abs=10*nS)
dend = Dendrite('dend', cm_abs=50*pF, gl_abs=2.5*nS)
dend.synapse('AMPA', tag='x', g=3*nS,  t_decay=2*ms)
dend.synapse('NMDA', tag='x', g=3*nS,  t_decay=60*ms)

In [ ]:
one_function_to_rule_them_all(soma, dend)

In [ ]:
print(soma.equations)

In [ ]:
soma_custom1 = Soma('soma', cm_abs=200*pF, gl_abs=10*nS)
print(soma_custom1.equations)

In [ ]:
custom_eqs = '''
dV_soma/dt = (gL_soma * (EL_soma-V_soma) + I_soma - w_soma) / C_soma  :volt
dw_soma/dt = (a_soma * (V_soma-EL_soma) - w_soma) / tauw_soma  :amp'''

soma_custom1.replace_equations(
    'dV_soma/dt = (gL_soma * (EL_soma-V_soma) + I_soma) / C_soma  :volt',
    custom_eqs)

print(soma_custom1.equations)

In [ ]:
one_function_to_rule_them_all(soma_custom1, dend)

In [ ]:
soma_custom2 = Soma('soma', cm_abs=200*pF, gl_abs=10*nS)
soma_custom2.replace_equations(
    "I_soma)",
    "I_soma - w_soma)")

soma_custom2.add_equations(
    'dw_soma/dt = (a_soma * (V_soma-EL_soma) - w_soma) / tauw_soma  :amp')

print(soma_custom2.equations)

In [ ]:
one_function_to_rule_them_all(soma_custom2, dend)

In [ ]:
print(dend.equations)

In [ ]:
dend_custom = Dendrite('dend', cm_abs=50*pF, gl_abs=2.5*nS)
print(dend_custom.equations)

In [ ]:
dend_custom.replace_equations(
    'I_dend = I_ext_dend  :amp',
    'I_dend = I_ext_dend + I_NMDA_x_dend + I_AMPA_x_dend  :amp'
    )

custom_synaptic_eqs = """
I_AMPA_x_dend = g_AMPA_x_dend * (E_AMPA-V_dend) * s_AMPA_x_dend * w_AMPA_x_dend  :amp
ds_AMPA_x_dend/dt = -s_AMPA_x_dend / t_AMPA_decay_x_dend  :1
I_NMDA_x_dend = g_NMDA_x_dend * (E_NMDA-V_dend) * s_NMDA_x_dend / (1 + Mg_con * exp(-Alpha_NMDA*(V_dend/mV+Gamma_NMDA)) / Beta_NMDA) * w_NMDA_x_dend  :amp
ds_NMDA_x_dend/dt = -s_NMDA_x_dend/t_NMDA_decay_x_dend  :1"""


dend_custom.add_equations(custom_synaptic_eqs)

print(dend_custom.equations)


In [ ]:
one_function_to_rule_them_all(soma, dend_custom)

In [ ]:
dend_custom_params = {
    't_AMPA_decay_x_dend': 2*ms,  # tau AMPA
    't_NMDA_decay_x_dend': 60*ms, # tau NMDA
    'g_AMPA_x_dend': 3*nS,        # AMPA conductance
    'g_NMDA_x_dend': 3*nS,        # NMDA conductance
    'w_NMDA_x_dend': 1,           # NMDA weight
    'w_AMPA_x_dend': 1,           # AMPA weight
    }
dend_custom.add_params(dend_custom_params)

one_function_to_rule_them_all(soma, dend_custom)

In [ ]:
# Create and customize soma
soma = Soma('soma', cm_abs=200*pF, gl_abs=10*nS)
soma.replace_equations("I_soma)", "I_soma - w)")
soma.add_equations('dw/dt = (a * (V_soma-EL_soma) - w) / tauw  :amp')

# Create and customize dendrite
dend = Dendrite('dend', cm_abs=50*pF, gl_abs=2.5*nS)
dend.replace_equations(
    'I_dend = I_ext_dend  :amp',
    'I_dend = I_ext_dend + I_NMDA + I_AMPA  :amp')
custom_synaptic_eqs = """
I_AMPA = g_AMPA * (E_AMPA-V_dend) * s_AMPA  :amp
ds_AMPA/dt = -s_AMPA / t_AMPA_decay  :1
I_NMDA = g_NMDA * (E_NMDA-V_dend) * s_NMDA / (1 + Mg_con * exp(-Alpha_NMDA*(V_dend/mV+Gamma_NMDA)) / Beta_NMDA)  :amp
ds_NMDA/dt = -s_NMDA/t_NMDA_decay  :1"""
dend.add_equations(custom_synaptic_eqs)

# Create neuron model and add parameters
model = NeuronModel([(soma, dend, 15*nS)], v_rest=-60*mV)
model.add_params({
        # spiking parameters
        'Vth': -40*mV,
        'tauw': 150*ms,
        'a': 0*nS,
        'b': 50*pA,
        'Vr': -50*mV,
        # synaptic parameters
        't_AMPA_decay': 2*ms,
        't_NMDA_decay': 60*ms,
        'g_AMPA': 3*nS,
        'g_NMDA': 3*nS})

# Create neuron group
neuron = model.make_neurongroup(
        1, method='euler',
        threshold='V_soma > Vth',
        reset='V_soma = Vr; w += b',
        refractory=4*ms)

# Create necessary Brian objects
poisson = b.PoissonGroup(10, rates=20*Hz)
syn = b.Synapses(poisson, neuron, on_pre='s_AMPA += 1; s_NMDA += 1')
syn.connect(p=1)
mon = b.StateMonitor(neuron, ['V_soma', 'V_dend', 'w'], record=True)
net = b.Network(neuron, poisson, syn, mon)

# Run simulation
b.start_scope()
b.seed(42)
net.run(500*ms)

# Plot results
time = mon.t / ms
v_soma = mon.V_soma[0] / mV
v_dend = mon.V_dend[0] / mV
w = mon.w[0] / pA

fig, (ax1, ax2) = b.subplots(2, 1, figsize=[6, 6], sharex=True)
ax1.plot(time, v_soma, label='soma')
ax1.plot(time, v_dend, label='dendrite', c='C3')
ax1.set_ylabel('Voltage (mV)')
ax1.legend()
ax2.plot(time, w, color='black', label='w')
ax2.set_ylabel('Adaptation current (pA)')
ax2.set_xlabel('Time (ms)')
ax2.legend()
fig.tight_layout()
b.show()